In [78]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import string
from bs4 import BeautifulSoup
max_features = 20000
maxlen = 80 # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [79]:
imdb = pd.read_csv('imdb.csv')

In [80]:
def clean(html):
    soup = BeautifulSoup(html)
    all_text = ''.join(soup.findAll(text=True))
    s = "123456789" + string.punctuation
    result = ''.join([i if i not in s else ' ' for i in all_text])
    result=result.replace("  "," ")
    return result
imdb.review = imdb.review.apply(clean)
imdb.sentiment = pd.Categorical(imdb.sentiment)
imdb.sentiment = imdb.sentiment.cat.codes

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imdb.review, imdb.sentiment, test_size=0.2, random_state=42)

In [85]:
tokenizer = Tokenizer(num_words=5000) #Define a Keras tokenizer; 
from keras.preprocessing.text import Tokenizer
tokenizer.fit_on_texts(X_train) # Fit the tokenizer on the text
X_train = tokenizer.texts_to_sequences(X_train) # Convert the text to sequences
X_test = tokenizer.texts_to_sequences(X_test)

In [90]:
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)
model = Sequential()
# keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)
model.add(Embedding(max_features, 100, input_shape=(maxlen,),trainable=True))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
from keras.utils import plot_model
plot_model(model, to_file='model.png')
es = EarlyStopping(monitor='val_loss', mode='min', patience=10)
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('Train...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=5, validation_split=0.1, callbacks=[es])
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 80, 100)           2000000   
_________________________________________________________________
flatten_5 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1024128   
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 3,024,257
Trainable params: 3,024,257
Non-trainable params: 0
_________________________________________________________________
None
Train...


/Users/pengfei/miniconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36000 samples, validate on 4000 samples
Epoch 1/5
36000/36000 [==============================] - 31s 865us/step - loss: 0.4163 - accuracy: 0.8008 - val_loss: 0.3702 - val_accuracy: 0.8357
Epoch 2/5
36000/36000 [==============================] - 30s 822us/step - loss: 0.1717 - accuracy: 0.9340 - val_loss: 0.5225 - val_accuracy: 0.8110
Epoch 3/5
36000/36000 [==============================] - 30s 827us/step - loss: 0.0309 - accuracy: 0.9886 - val_loss: 0.8918 - val_accuracy: 0.8020
Epoch 4/5
36000/36000 [==============================] - 30s 839us/step - loss: 0.0219 - accuracy: 0.9920 - val_loss: 0.9833 - val_accuracy: 0.7980
Epoch 5/5
10000/10000 [==============================] - 0s 44us/step


In [91]:
acc

0.8026000261306763